In [1]:
import os

import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
import lib.metrics
import lib.load_data
import lib.evaluate

import archs.unet
import archs.multiresunet
import archs.attentionunet
import archs.nestedunet
import archs.iterlunet

from archs.unet import UNet
from archs.multiresunet import MultiResUnet
from archs.nestedunet import NestedUNet
from archs.attentionunet import AttUNet
from archs.iterlunet import IterLUNet

from lib.evaluate import testModel
from lib.load_data import get_data
import cv2
from skimage import io
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import matplotlib.font_manager
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, model_from_json

print('TensorFlow version: {version}'.format(version=tf.__version__))
print('Keras version: {version}'.format(version=tf.keras.__version__))
print('Eager mode enabled: {mode}'.format(mode=tf.executing_eagerly()))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

TensorFlow version: 2.4.1
Keras version: 2.4.0
Eager mode enabled: True
Num GPUs Available:  0
Num CPUs Available:  1


In [2]:
def prediction(test_ids, image_path, mask_path, unet, multiresunet, attnunet, nestedunet, proposed):
    ids = test_ids
    # empty list to store results
    image_names, mask1, mask2, mask3, mask4, mask5,image_id, mask_id, has_mask = [], [], [], [], [], [],[], [], []
    
    image_path = os.path.abspath(image_path)
    mask_path = os.path.abspath(mask_path)
    #itetrating through each image in test data
    for n, i in tqdm(enumerate(ids), total=len(ids)):
        filename = i
        image_names.append(filename)
        #i = i.split('/')[-1]
        i = image_path + '/' + i

        #Creating a empty array of shape 1,256,256,1
        X = np.empty((1,256,256,3))
        # read the image
        img = io.imread(i)
        #resizing the image and coverting them to array of type float64
        img = cv2.resize(img, (256,256))
        img = np.array(img)
        
        # standardising the image
        #img -= img.mean()
        #img /= img.std()
        img = img / 255
        #converting the shape of image from 256,256,3 to 1,256,256,3
        X[0,] = img
        
        #make prediction of mask
        predict1 = unet.predict(X)
        predict2 = multiresunet.predict(X)
        predict3 = attnunet.predict(X)
        predict4 = nestedunet.predict(X)
        predict5 = proposed.predict(X)
        
        image_id.append(image_path + '/' + filename)
        mask_id.append(mask_path + '/' + format(filename.split('.')[0]+'.png'))
        has_mask.append(1)
    
        mask1.append(predict1)
        mask2.append(predict2)
        mask3.append(predict3)
        mask4.append(predict4)
        mask5.append(predict5)
            
            
    return pd.DataFrame({'file_name':image_names,'image_path': image_id, 'mask_path': mask_id,  'UNet_mask': mask1, 'MultiresUNet_mask': mask2, 'AttentionUNet_mask': mask3,
                         'NestedUNet_mask': mask4, 'IterLUNet_mask':mask5, 'has_mask': has_mask})

In [3]:
def iou_metric(y_true_in, y_pred_in):
    smooth = 1e-15
    intersection = y_true_in.ravel() * y_pred_in.ravel()
    union = y_true_in.ravel() + y_pred_in.ravel() - intersection

    iou = ((np.sum(intersection) + smooth)/(np.sum(union) + smooth))
    return iou

In [4]:
#visualizing prediction
img_height = 256
img_width = 256
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.transform import resize


def show_visualization(df_pred,experiment_name):
    count = 0
    fig, axs = plt.subplots(8,7, figsize=(48, 62))
    
    
    #df_pred = df_pred.sample(frac = 0.8)
    #df_pred = df_pred.sample(n=len(df_pred), random_state=2022)
    for i in range(len(df_pred)):
        if df_pred.has_mask[i]==1 and count<8:
            
            #read levee crack image
            img = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img = cv2.resize(img, (256,256))
            #img = np.array(img, dtype = np.float64)
            axs[count][0].imshow(img)
            axs[count][0].set_axis_off() 
            axs[count][0].set_title('Original Image ' + str(count+1), fontsize=56)
    
    
            
            #read original mask and overlay original mask with levee crack image    
            mask = cv2.imread(df_pred.mask_path[i], cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (256, 256))
            
            img[mask==255] = (255,0,0)
            axs[count][1].imshow(img)
            axs[count][1].set_axis_off()
            axs[count][1].set_title('Ground Truth '+ str(count+1), fontsize=56)
            mask = np.array(mask, dtype = np.float64)
            mask = mask / 255
            
        
            
            #pred_unet = np.array(df_pred.UNet_mask[i]).squeeze().round()
            pred_unet = np.array(df_pred.UNet_mask[i])
            iou_unet = iou_metric(mask, pred_unet )
            pred_unet = np.round(pred_unet,0).squeeze()
            img1 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img1 = cv2.resize(img1, (256,256))
            img1 = img1.squeeze()
            img1[pred_unet==1] = (0,0,0)
            axs[count][2].imshow(img1)
            axs[count][2].set_axis_off()
            axs[count][2].set_title('[IoU M1=' + str(round(iou_unet,2)) + ']', fontsize=56)
            
            
            
            #pred_multiresunet = np.array(df_pred.MultiresUNet_mask[i]).squeeze().round()
            pred_multiresunet = np.array(df_pred.MultiresUNet_mask[i])
            iou_multiresunet = iou_metric(mask, pred_multiresunet )
            pred_multiresunet = np.round(pred_multiresunet,0).squeeze()
            img2 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img2 = cv2.resize(img2, (256,256))
            img2 = img2.squeeze() / 255
            img2[pred_multiresunet==1] = (0,0,0)
            axs[count][3].imshow((img2 * 255).astype(np.uint8))
            axs[count][3].set_axis_off()
            axs[count][3].set_title('[IoU M2=' + str(round(iou_multiresunet, 2)) + ']', fontsize=56)
            
            
            
            
            #pred_attentionunet = np.array(df_pred.AttentionUNet_mask[i]).squeeze().round()
            pred_attentionunet = np.array(df_pred.AttentionUNet_mask[i])
            iou_attentionunet = iou_metric(mask, pred_attentionunet )
            pred_attentionunet = np.round(pred_attentionunet,0).squeeze()
            img3 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img3 = cv2.resize(img3, (256,256))
            img3 = img3.squeeze() / 255
            img3[pred_attentionunet==1] = (0,0,0)
            axs[count][4].imshow((img3 * 255).astype(np.uint8))
            axs[count][4].set_axis_off() 
            axs[count][4].set_title('[IoU M3=' + str(round(iou_attentionunet, 2)) + ']', fontsize=56)
            
            
            #pred_nestedunet = np.array(df_pred.NestedUNet_mask[i]).squeeze().round()
            pred_nestedunet = np.array(df_pred.NestedUNet_mask[i])
            iou_nestedunet = iou_metric(mask, pred_nestedunet )
            pred_nestedunet = np.round(pred_nestedunet,0).squeeze()
            img4 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img4 = cv2.resize(img4, (256,256))
            img4 = img4.squeeze()
            img4[pred_nestedunet==1] = (0,0,0)
            axs[count][5].imshow(img4)
            axs[count][5].set_axis_off() 
            axs[count][5].set_title('[IoU M4=' + str(round(iou_nestedunet, 2)) + ']', fontsize=56)
            
            
            
            #pred_inception = np.array(df_pred.IterLUNet[i]).squeeze().round()
            pred_inception = np.array(df_pred.IterLUNet_mask[i])
            iou_inception = iou_metric(mask, pred_inception )
            pred_inception = np.round(pred_inception,0).squeeze()
            img5 = cv2.imread(df_pred.image_path[i], cv2.IMREAD_COLOR)
            img5 = cv2.resize(img5, (256,256))
            img5 = img5.squeeze() 
            img5[pred_inception==1] = (0,0,255)
            axs[count][6].imshow(img5)
            axs[count][6].set_axis_off() 
            axs[count][6].set_title('[IoU M5=' + str(round(iou_inception, 2)) + ']', fontsize= 56)
            
            
            fig.subplots_adjust(top=0.96)
            fig.tight_layout()
            #plt.suptitle('Results of all models on hold-out levee crack test images', fontweight='regular', fontsize=16)
            plt.savefig(str(experiment_name)  + '.png', format='png', facecolor="w", transparent=False)
        
            count +=1
        if (count==10):
            break
    fig.tight_layout() 
    plt.close()
    

In [5]:
#ata_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/exptl/code/Users/mpanta2/datasets/LeveeCrackDataset/augmentedLeveeCrack"
from sklearn.model_selection import train_test_split
img_height = 256
img_width = 256

path_test = "/home/mpanta1/workspace/deeplearning/image_segmentation/unet_architecture/IEEEAccess_code_test/IterLUNet/dataset/experiment_2/test"
test_images = sorted(next(os.walk(path_test + "/images"))[2])

X_test, y_test = get_data(test_images, path_test, img_height, img_width, train=True)

(26, 256, 256, 3)
Getting and resizing images ... 


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:01<00:00, 22.83it/s]

Done!
(26, 256, 256, 3)


In [6]:
path_test= "../dataset/experiment_2/deepcrack_test_data/"
test_images_deepcrack = (next(os.walk(path_test + "/images"))[2])

deepcrack_path_test = "../dataset/experiment_2/deepcrack_test_data"
test_images = sorted(next(os.walk(deepcrack_path_test + "/images"))[2])

X_test_DC, y_test_DC = get_data(test_images, deepcrack_path_test, img_height, img_width, train=True)

(237, 256, 256, 3)
Getting and resizing images ... 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:07<00:00, 33.51it/s]

Done!
(237, 256, 256, 3)


In [7]:
tf.keras.backend.clear_session()
%time
print('IterLUNet model loaded')
inception_64_model = IterLUNet(input_filters=64, height=img_height, width=img_width, n_channels=3)
inception_64_model.load_weights("../models/experiment_2/iterlunet.h5")
results_levee = testModel(inception_64_model, X_test, y_test, 4, 'iterlunet_levee')
results_deepcrack = testModel(inception_64_model, X_test_DC, y_test_DC, 4, 'iterlunet_deepcrack')

2022-09-13 21:40:43.315520: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


CPU times: user 48 µs, sys: 51 µs, total: 99 µs
Wall time: 87.5 µs
IterLUNet model loaded


2022-09-13 21:40:47.198028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-13 21:40:47.198779: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2300000000 Hz


7/7 [==============================] - 52s 7s/step
0.72641695 0.39532253 0.5120065
Test Jacard Index : 0.35493225824922614
Test Dice Coefficient : 0.458476310104759
Test iou_all :0.36373677387961173
60/60 [==============================] - 439s 7s/step
0.74644953 0.73572195 0.7410469
Test Jacard Index : 0.580033763600379
Test Dice Coefficient : 0.7245992568816298
Test iou_all :0.5850898225966435


In [8]:
tf.keras.backend.clear_session()
%time
print('UNet model loaded')
UNet_model = UNet(input_filters=32, height=img_height, width=img_width, n_channels=3)
UNet_model.load_weights("../models/experiment_2/unet.hdf5")
results_levee = testModel(UNet_model, X_test, y_test, 4, 'unet_levee')
results_deepcrack = testModel(UNet_model, X_test_DC, y_test_DC, 4, 'unet_deepcrack')

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 68.4 µs
UNet model loaded
7/7 [==============================] - 4s 507ms/step
0.64436466 0.37728974 0.47591865
Test Jacard Index : 0.28156021390619385
Test Dice Coefficient : 0.41422363555919184
Test iou_all :0.2846818394142164
60/60 [==============================] - 31s 518ms/step
0.813713 0.51647717 0.6318859
Test Jacard Index : 0.436969927360464
Test Dice Coefficient : 0.5843404320748554
Test iou_all :0.442445808655636


In [9]:
tf.keras.backend.clear_session()
%time
print('MultiResUnet model loaded')
MultiResUnet_model = MultiResUnet(input_filters=32, height=img_height, width=img_width, n_channels=3)
MultiResUnet_model.load_weights("../models/experiment_2/multiresunet.hdf5")
results_levee = testModel(MultiResUnet_model, X_test, y_test, 4, 'multiresunet_levee')
results_deepcrack = testModel(MultiResUnet_model, X_test_DC, y_test_DC, 4, 'multiresunet_deepcrack')

CPU times: user 17 µs, sys: 14 µs, total: 31 µs
Wall time: 69.9 µs
MultiResUnet model loaded
7/7 [==============================] - 8s 1s/step
0.704641 0.34656984 0.46462098
Test Jacard Index : 0.25031778027742135
Test Dice Coefficient : 0.369477722758654
Test iou_all :0.25220099169174737
60/60 [==============================] - 70s 1s/step
0.8839493 0.43529186 0.5833292
Test Jacard Index : 0.39524455306982526
Test Dice Coefficient : 0.5422778033412644
Test iou_all :0.40073931490153697


In [10]:
tf.keras.backend.clear_session()
%time
print('AttUNet model loaded')
AttUNet_model = AttUNet(input_filters=32, height=img_height, width=img_width, n_channels=3)
AttUNet_model.load_weights("../models/experiment_2/attentionunet.hdf5")
results_levee = testModel(AttUNet_model, X_test, y_test, 4, 'attentionunet_levee')
results_deepcrack = testModel(AttUNet_model, X_test_DC, y_test_DC, 4, 'attentionunet_deepcrack')

CPU times: user 24 µs, sys: 18 µs, total: 42 µs
Wall time: 84.4 µs
AttUNet model loaded
7/7 [==============================] - 7s 981ms/step
0.626786 0.403742 0.49112657
Test Jacard Index : 0.280135356013409
Test Dice Coefficient : 0.4158171381882567
Test iou_all :0.28149163506369024
60/60 [==============================] - 53s 892ms/step
0.7705007 0.5613572 0.64950794
Test Jacard Index : 0.420646290758331
Test Dice Coefficient : 0.561999287456679
Test iou_all :0.4241672301758641


In [11]:
tf.keras.backend.clear_session()
%time
print(f'NestedUNet model loaded')
NestedUNet_model = NestedUNet(input_filters=32, height=img_height, width=img_width, n_channels=3)
NestedUNet_model.load_weights("../models/experiment_2/nested_unet.hdf5")
results_levee = testModel(NestedUNet_model, X_test, y_test, 4, 'nestedunet_levee')
results_deepcrack = testModel(NestedUNet_model, X_test_DC, y_test_DC, 4, 'nestedunet_deepcrack')

CPU times: user 0 ns, sys: 64 µs, total: 64 µs
Wall time: 138 µs
NestedUNet model loaded
7/7 [==============================] - 15s 2s/step
0.66347206 0.400294 0.49932763
Test Jacard Index : 0.2934889376291993
Test Dice Coefficient : 0.42778055368610146
Test iou_all :0.29654142693317004
60/60 [==============================] - 140s 2s/step
0.8149447 0.5283849 0.6411002
Test Jacard Index : 0.4514945620754344
Test Dice Coefficient : 0.5973044892044579
Test iou_all :0.4569336542632153


In [ ]:
img_height = 256
img_width = 256

path_test= "../dataset/experiment_1/test/"
test_images_leveecrack = (next(os.walk(path_test + "/images"))[2])

image_leveecrack_path = "../dataset/experiment_1/test/images"
mask_leveecrack_path = "../dataset/experiment_1/test/masks"

In [ ]:
df_all_preds_all = []
df_all_preds_all = prediction(test_images_leveecrack, os.path.abspath(image_leveecrack_path), os.path.abspath(mask_leveecrack_path), UNet_model, MultiResUnet_model, AttUNet_model, NestedUNet_model, inception_64_model)

In [ ]:
df_for_visualization = df_all_preds_all.sample(frac = 0.98)
show_visualization(df_for_visualization, "ieee_access_final")

## Inference for DeepCrack Test Data

In [ ]:
img_height = 256
img_width = 256

path_test= "../dataset/experiment_2/deepcrack_test_data/"
test_images_deepcrack = (next(os.walk(path_test + "/images"))[2])

image_deepcrack_path = "../dataset/experiment_2/deepcrack_test_data/images"
mask_deepcrack_path = "../dataset/experiment_2/deepcrack_test_data/masks"

In [ ]:
df_all_preds_all = []
df_all_preds_all = prediction(test_images_deepcrack, os.path.abspath(image_deepcrack_path), os.path.abspath(mask_deepcrack_path), UNet_model, MultiResUnet_model, AttUNet_model, NestedUNet_model, inception_64_model)

In [ ]:
df_for_visualization = df_all_preds_all.sample(frac = 0.98)
show_visualization(df_for_visualization, "ieee_access_deepcrack_test_images")